In [134]:
from pathlib import Path
from xml.dom import minidom
import numpy as np
import pandas as pd

import os
import filecmp

from bs4 import BeautifulSoup

ImportError: cannot import name 'round' from 'math' (unknown location)

In [2]:
path = Path("c:\\Users\\fe0968\\Documents\\data\\medaka\\landmarks\\workshop_landmarks_selected\\")

In [3]:
path_alex = path / 'pointsets_alexey'
path_TT = path / 'Pointsets_803_4_TT'
path_JB = path / 'Pointsets_JB'
path_JB2 = path / 'Pointsets_JB2'

In [121]:
set_vert = {
    'name': 'Vert',
    'file_name': 'PointSet1_Vert'.lower(),
    'landmarks': [
        'Transition skull to spine',
        'Vert1',
        'Vert2',
        'Vert3',
        'Vert4',
        'Vert5',
        'Vert_Last_Center'
    ]
}

set_fins = {
    'name': 'Fins',
    'file_name': 'PointSet2_Fins'.lower(),
    'landmarks': [
        'Pectoral_dorsal most breast fin to body connection 1_right',
        'Pectoral_dorsal most breast fin to body connection 2_left',
        'Abdominal_fins back 1_right',
        'Abdominal_fins back 2_left'
    ]
}

set_digest = {
    'name': 'Digest',
    'file_name': 'PointSet3_Digest'.lower(),
    'landmarks': [
        'anus_Center',
        'esophagus'
    ]
}

set_heart = {
    'name': 'Heart',
    'file_name': 'PointSet4_Heart'.lower(),
    'landmarks': [
        'tip of bulbus arteriosus vessel inside',
        'sinus venosus',
        'apex of ventricle',
        'anterior most point of ventricle'
    ]
}

set_eyes = {
    'name': 'Eyes',
    'file_name': 'PointSet5_Eyes'.lower(),
    'landmarks': [
        'optic nerve head 1_right',
        'optic nerve head 2_left',
        'optic chiasm_crossing',
        'most_anterior_right',
        'most_anterior_lef',
        'most_posterior_right',
        'most_posterior_left',
        'most_dorsal_right',
        'most_dorsal_left',
        'most_ventral_right',
        'most_ventral_left'
    ]
}

set_skull_front = {
    'name': 'Skull Front',
    'file_name': 'PointSet6_Skull_Front'.lower(),
    'landmarks': [
        'ventral side of nostril outlet right',
        'ventral side of nostril outlet left',
        'dorsal side of nostril outlet right',
        'dorsal side of nostril outlet left',
        'mandible dentary',
        'tongue tip',
        'upper jaw channel',
        'hyoid fusion'
    ]
}

set_skull_center = {
    'name': 'Skull Center',
    'file_name': 'PointSet7_Skull_Center'.lower(),
    'landmarks': [
        'subhypophysis bone',
        'hyoid between branchial arches',
        'split of afferent branchial artery 1',
        'split of afferent branchial artery 2',
        'split of afferent branchial artery 3',
        'gills bone right',
        'gills bone left'
    ]
}

set_skull_end = {
    'name': 'Skull End',
    'file_name': 'PointSet8_Skull_End'.lower(),
    'landmarks': [
        'skull landmark A right',
        'skull landmark A left',
        'transition skull to spine',
        'fusion of epibranchial artery 2',
        'center of utricle right',
        'center of utricle left'
    ]
}

set_brain = {
    'name': 'Brain',
    'file_name': 'PointSet9_Brain'.lower(),
    'landmarks': [
    'hypophysis',
    'olfactoryN_right',
    'olfactoryN_left',
    'glomerulosus_R',
    'glomerulosus_L',
    'OT_rightmost',
    'OT_leftmost',
    'cerebellum',
    'OT cerebellum torus',
    'epiphysis'
    ]
}

all_landmarks = [set_vert, set_fins, set_digest, set_heart, set_eyes, set_skull_front, set_skull_center, set_skull_end, set_brain]

landmarks_pointset_names = [x['file_name'] for x in all_landmarks]

print(landmarks_pointset_names)

['pointset1_vert', 'pointset2_fins', 'pointset3_digest', 'pointset4_heart', 'pointset5_eyes', 'pointset6_skull_front', 'pointset7_skull_center', 'pointset8_skull_end', 'pointset9_brain']


In [62]:
def read_landmarks(file_name, mode='def'):
    
    if mode == 'lowercase':
        file_name = file_name.lower()
    
    with open(str(file_name), 'r') as f:
        data = f.read()
    
    xml_data = BeautifulSoup(data, "xml")
    points = xml_data.find_all('point')
    
    landmarks = []

    for p in points:
        if p.find('x').text == '0' and p.find('y').text == '0' and p.find('z').text == '0':
            continue

        x = float(p.find('x').text)
        y = float(p.find('y').text)
        z = float(p.find('z').text)

        landmarks.append(np.asarray([x,y,z]))
        
    f.close()
        
    return landmarks

def get_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def print_landmark_comparison(landmark_set, landmark1, landmark2, landmark3, landmark4):
     
    #print('------------------------------------------------------------------------')
    print(landmark_set['name'])
    #print('------------------------------------------------------------------------')
    data = {'landmark': landmark_set['landmarks'],
           'AE vs TT': [get_distance(landmark1[i], landmark2[i]) for i in range(len(landmark_set['landmarks']))],
            'AE vs JB': [get_distance(landmark1[i], landmark3[i]) for i in range(len(landmark_set['landmarks']))],
            'TT vs JB': [get_distance(landmark2[i], landmark3[i]) for i in range(len(landmark_set['landmarks']))],
            'JB vs JB2': [get_distance(landmark3[i], landmark4[i]) for i in range(len(landmark_set['landmarks']))]
           }
    
    df = pd.DataFrame(data)
    return df

    ##print(df)
    ##print()
    ##print(df.describe())
    
def read_landmarks_all_three(landmark_set):
    land_ae = read_landmarks(path_alex / landmark_set['file_name'])
    land_tt = read_landmarks(path_TT / landmark_set['file_name'])
    land_jb = read_landmarks(path_JB / landmark_set['file_name'])
    land_jb2 = read_landmarks(path_JB2 / landmark_set['file_name'])

    return land_ae, land_tt, land_jb, land_jb2

def show_results(i):
    land_ae, land_tt, land_jb, land_jb2 = read_landmarks_all_three(all_landmarks[i])
    df = print_landmark_comparison(all_landmarks[i], land_ae, land_tt, land_jb, land_jb2)
    return df.round(1).style.pipe(make_pretty)
    return df.round(1).style.pipe(make_pretty)

def make_pretty(styler):
    #styler.set_caption("Weather Conditions")
    #styler.format(rain_condition)
    #styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=0, vmax=20, cmap="coolwarm")
    #styler.background_gradient(axis=None, cmap="coolwarm")
    return styler
    

def fix_files(file_list, old_name, new_name):
    
    if old_name in file_list:
        file_list[file_list.index(old_name)] = new_name
        if os.path.exists(data_path / old_name):
            os.rename(data_path / old_name, data_path / new_name)
    return file_list
    
def print_landmarks_count_for_pointsets():
    for land in all_landmarks:
        print(f"{land['file_name']}: {len(land['landmarks'])} landmarks")
    

In [80]:
print_landmarks_count_for_pointsets()

pointset1_vert: 7 landmarks
pointset2_fins: 4 landmarks
pointset3_digest: 2 landmarks
pointset4_heart: 4 landmarks
pointset5_eyes: 11 landmarks
pointset6_skull_front: 8 landmarks
pointset7_skull_center: 7 landmarks
pointset8_skull_end: 6 landmarks
pointset9_brain: 10 landmarks


## Landmark Consistancy

In [140]:
show_results(0)

In [233]:
show_results(1)

Fins


,landmark,AE vs TT,AE vs JB,TT vs JB,JB vs JB2
0,Pectoral_dorsal most breast fin to body connection 1_right,8.800000,139.700000,136.900000,2.200000
1,Pectoral_dorsal most breast fin to body connection 2_left,12.100000,140.900000,140.400000,2.100000
2,Abdominal_fins back 1_right,15.600000,12.100000,6.500000,5.400000
3,Abdominal_fins back 2_left,7.500000,2.000000,5.800000,11.100000


In [234]:
show_results(2)

Digest


,landmark,AE vs TT,AE vs JB,TT vs JB,JB vs JB2
0,anus_Center,0.800000,15.500000,14.900000,1.200000
1,esophagus,4.900000,1.400000,5.300000,3.200000


In [235]:
show_results(3)

Heart


,landmark,AE vs TT,AE vs JB,TT vs JB,JB vs JB2
0,tip of bulbus arteriosus vessel inside,10.800000,1.100000,10.900000,1.800000
1,sinus venosus,54.700000,11.300000,45.700000,40.300000
2,apex of ventricle,7.000000,30.200000,37.000000,37.800000
3,anterior most point of ventricle,1.300000,3.200000,3.700000,0.200000


In [236]:
show_results(4)

Eyes


,landmark,AE vs TT,AE vs JB,TT vs JB,JB vs JB2
0,optic nerve head 1_right,4.100000,1.400000,5.500000,7.200000
1,optic nerve head 2_left,2.000000,6.300000,8.100000,8.200000
2,optic chiasm_crossing,6.200000,2.000000,7.100000,2.200000
3,most_anterior_right,7.600000,5.700000,1.900000,8.100000
4,most_anterior_lef,3.300000,6.300000,7.000000,7.800000
5,most_posterior_right,5.600000,1.100000,4.600000,1.300000
6,most_posterior_left,6.500000,3.200000,3.800000,2.900000
7,most_dorsal_right,2.900000,6.400000,6.800000,1.300000
8,most_dorsal_left,1.700000,7.300000,6.700000,3.000000
9,most_ventral_right,8.500000,4.100000,7.100000,8.700000


In [237]:
show_results(5)

Skull Front


,landmark,AE vs TT,AE vs JB,TT vs JB,JB vs JB2
0,ventral side of nostril outlet right,5.700000,3.900000,2.400000,1.700000
1,ventral side of nostril outlet left,1.000000,1.500000,1.600000,0.200000
2,dorsal side of nostril outlet right,1.200000,2.300000,1.500000,2.100000
3,dorsal side of nostril outlet left,2.200000,1.800000,2.300000,0.900000
4,mandible dentary,0.800000,11.700000,12.100000,3.700000
5,tongue tip,0.800000,1.000000,0.300000,1.000000
6,upper jaw channel,7.900000,6.700000,1.500000,3.300000
7,hyoid fusion,4.300000,3.600000,0.900000,3.200000


In [238]:
show_results(6)

Skull Center


,landmark,AE vs TT,AE vs JB,TT vs JB,JB vs JB2
0,subhypophysis bone,7.100000,8.600000,1.900000,5.200000
1,hyoid between branchial arches,2.200000,1.700000,0.800000,11.900000
2,split of afferent branchial artery 1,4.700000,3.600000,7.200000,2.100000
3,split of afferent branchial artery 2,3.700000,3.300000,6.800000,2.700000
4,split of afferent branchial artery 3,20.700000,4.000000,19.700000,2.000000
5,gills bone right,3.000000,1.000000,3.800000,4.000000
6,gills bone left,3.300000,1.300000,4.100000,4.100000


In [240]:
show_results(7)

Skull End


,landmark,AE vs TT,AE vs JB,TT vs JB,JB vs JB2
0,skull landmark A right,1.400000,2.700000,3.700000,14.700000
1,skull landmark A left,2.900000,2.100000,1.100000,17.400000
2,transition skull to spine,2.000000,3.400000,4.900000,3.700000
3,fusion of epibranchial artery 2,3.700000,11.400000,9.600000,3.700000
4,center of utricle right,9.100000,9.100000,1.200000,1.300000
5,center of utricle left,8.500000,8.300000,0.500000,1.200000


## Testing

In [6]:
data_path = Path('c:\\Users\\fe0968\\Documents\\Medaka\\inbredpanel_pointsets_v2\\inbredpanel_pointsets\\')

In [115]:
files = os.listdir(data_path)
files = [f for f in files if os.path.isfile(data_path / f)]
len(files)

2120

In [116]:
# Filtering 1: Delete duplicates. Files with (1) in the name
duplicates = []
same_name_diff_content = []

for f in files:
    if '(1)' in f:
        if filecmp.cmp(data_path / f, data_path / f.replace(' (1)', '')):
            duplicates.append(f)
        else:
            same_name_diff_content.append(f)
            
            
print('Duplicates:', len(duplicates))
        
        
filtered_files = [x for x in files if (x not in duplicates)]
print('New:', len(filtered_files))

print()
print('-----------------------------------------------')
print('WARNING. Same name, different content', same_name_diff_content)
print('-----------------------------------------------')

filtered_files = [x for x in filtered_files if (x not in same_name_diff_content)]
print('New:', len(filtered_files))

Duplicates: 80
New: 2040

-----------------------------------------------
WARNING. Same name, different content ['1064_pointset1_vert_JO (1).mps']
-----------------------------------------------
New: 2039


In [117]:
manual_exceptions = ['PointSet3_Digest.mps', 'PointSet4_Heart.mps', 'PointSet9_brain.mps', '1370_pointset2_fins.mps', '961_pointset2_fins.mps', '954_pointset9_brain.mps']
manual_exceptions = [x.lower() for x in manual_exceptions]

filtered_files = [x for x in filtered_files if (x.lower() not in manual_exceptions)]
print('New:', len(filtered_files))

New: 2033


In [118]:
filtered_files = fix_files(filtered_files, '1265PointSet3_Digest_VC.mps', '1265_PointSet3_Digest_VC.mps')
filtered_files = fix_files(filtered_files, '1274pointset6_skull_front_JO.mps', '1274_pointset6_skull_front_JO.mps')
filtered_files = fix_files(filtered_files, '1300_ointSet3_Digest_VC.mps', '1300_PointSet3_Digest_VC.mps')
filtered_files = fix_files(filtered_files, '1300PointSet4_Heart_VC.mps', '1300_PointSet4_Heart_VC.mps')
filtered_files = fix_files(filtered_files, '1300_PointSet2_Vert_CS.mps', '1300_PointSet1_Vert_CS.mps')
filtered_files = fix_files(filtered_files, '876_PointSet3-Digest_VC.mps', '876_PointSet3_Digest_VC.mps')

In [127]:
samples = []
points = []
names = []

for f in filtered_files:
    
    s = f.split('_')
    samples.append(s[0])
    names.append(s[-1][:-4].lower())
    #points.append(s[1])
    points.append(f.replace(f'{s[0]}_', '').replace(f'_{s[-1]}', '').lower())
    
data = {'sample': samples,
        'point_set': points,
       'name': names}


print('Unique samples:', len(set(samples)))
print(set(samples))

print()

print('Unique pointsets:', len(set(points)))
print(set(points))

print()

print('Unique names:', len(set(names)))
print(set(names))
    

Unique samples: 181
{'1404', '814', '1369', '947', '956', '1362', '803', '880', '818', '1257', '1346', '1297', '846', '509', '475', '805', '1105', '1264', '1366', '1380', '511', '856', '547', '1075', '800', '425', '1104', '1232', '500', '1230', '1227', '535', '946', '471', '1391', '1293', '804', '463', '1081', '807', '1134', '1285', '853', '1219', '454', '528', '1310', '531', '1274', '955', '979', '1337', '1291', '508', '1222', '1273', '1217', '582', '961', '871', '876', '1129', '1321', '891', '909', '1256', '1115', '841', '972', '1265', '1305', '1417', '488', '960', '1393', '1338', '1192', '578', '883', '1376', '417', '816', '978', '819', '1295', '1248', '828', '1284', '1131', '913', '811', '565', '1292', '1341', '906', '971', '489', '470', '824', '1400', '1399', '444', '1100', '954', '526', '838', '1389', '1106', '1191', '1082', '502', '1405', '831', '423', '1184', '1064', '854', '919', '1394', '1383', '1418', '537', '561', '1395', '801', '1119', '1353', '1401', '1296', '879', '893',

In [122]:
print(landmarks_pointset_names)

print()
print('-----------------------------------------------')
print('WARNING: Problematic naming')
print('-----------------------------------------------')
print([p for p in points if p not in landmarks_pointset_names])

print('Based on pointset names')
print([samples[points.index(p)] + '_' + points[points.index(p)] + '_' + names[points.index(p)]   for p in points if p not in landmarks_pointset_names])


print('Based on names')
print([samples[names.index(p)] + '_' + points[names.index(p)] + '_' + names[names.index(p)]   for p in names if p not in participants])
    
# TODO: Make a list of exact file names to check manually


['pointset1_vert', 'pointset2_fins', 'pointset3_digest', 'pointset4_heart', 'pointset5_eyes', 'pointset6_skull_front', 'pointset7_skull_center', 'pointset8_skull_end', 'pointset9_brain']

-----------------------------------------------
-----------------------------------------------
[]
Based on pointset names
[]
Based on names
[]


In [123]:
people = set(names)
print(people)

print('-----------------------------------------------')
print('WARNING: Problematic names')
print('-----------------------------------------------')
for p in people:
    if p not in participants:
        print('Problematic name: ', p)
        print('Count:', names.count(p))
        print([samples[i] + '_' + points[i] + '_' + names[i] + '.mps' for i, x in enumerate(names) if x == p])

{'jvm', 'bew', 'tt', 'vc', 'ca', 'jo', 'kp', 'cs', 'ttt', 'jf', 'ra', 'tc', 'kk'}
-----------------------------------------------
-----------------------------------------------


In [96]:
[i for i, x in enumerate(names) if x == 'fins']

[799, 1740]

## Make a data frame

In [128]:
df = pd.DataFrame(data)

## Participants stats

In [156]:
participants = ['ca', 'jf', 'tc', 'cs', 'ra', 'kk', 'vc', 'bew', 'ttt', 'kp', 'jo', 'jvm', 'tt']

participants_names = {'jf': 'Jana',
                     'jo': 'Jasmin',
                     'bew': 'Bettina',
                     'ca': 'Cassian',
                     'cs': 'Christina',
                     'jvm': 'Javier',
                     'kk': 'Kristaps',
                     'kp':'Kaisa',
                     'ra':'Rashi',
                     'tc':'???',
                     'tt': 'Thomas',
                     'ttt': 'Tina',
                     'vc': 'Valerie'}

participants_pointset_count= {
                     'jf': 3,
                     'jo': 5,
                     'bew': 4,
                     'ca': 4,
                     'cs': 3,
                     'jvm': 3,
                     'kk': 3,
                     'kp': 5,
                     'ra': 4,
                     'tc': 2,
                     'tt': 3,
                     'ttt': 3,
                     'vc': 5}

print('Participants')
res = df.groupby(['name']).count()
res

Participants


,sample,point_set
name,,
bew,120,120
ca,160,160
cs,177,177
jf,120,120
jo,299,299
jvm,180,180
kk,180,180
kp,108,108
ra,160,160


In [167]:
print('Current status')
for name in res.index:
    print(f"{participants_names[name]} ({name}): {round(res.loc[name]['point_set'] / (participants_pointset_count[name]*60) * 100 ,1)}% done")
    #print(res.loc[name]['point_set'])

Current status
Bettina (bew): 50.0% done
Cassian (ca): 66.7% done
Christina (cs): 98.3% done
Jana (jf): 66.7% done
Jasmin (jo): 99.7% done
Javier (jvm): 100.0% done
Kristaps (kk): 100.0% done
Kaisa (kp): 36.0% done
Rashi (ra): 66.7% done
??? (tc): 100.0% done
Thomas (tt): 71.7% done
Tina (ttt): 66.7% done
Valerie (vc): 53.3% done


In [155]:
df[df['name'] == 'tc']['point_set'].unique()

array(['pointset1_vert', 'pointset2_fins'], dtype=object)

## Pointset stats

In [139]:
total_samples = 74*3

for p in landmarks_pointset_names:
    total = len(df[df['point_set'] == p]['sample'])
    uniq = len(df[df['point_set'] == p]['sample'].unique())
    print(f"{p}: total {total}, unique {uniq}. finished: {round(total / (total_samples * 2) * 100, 1)}")

pointset1_vert: total 322, unique 180. finished: 72.5
pointset2_fins: total 319, unique 181. finished: 71.8
pointset3_digest: total 270, unique 160. finished: 60.8
pointset4_heart: total 270, unique 160. finished: 60.8
pointset5_eyes: total 160, unique 160. finished: 36.0
pointset6_skull_front: total 241, unique 160. finished: 54.3
pointset7_skull_center: total 142, unique 120. finished: 32.0
pointset8_skull_end: total 141, unique 120. finished: 31.8
pointset9_brain: total 168, unique 159. finished: 37.8


In [132]:
print(len(df[df['point_set'] == 'pointset9_brain']['sample'].unique()))
print(len(df[df['point_set'] == 'pointset9_brain']['sample']))

159
168


In [44]:
print_landmarks_count_for_pointsets()

pointset1_vert: 6 landmarks
pointset2_fins: 4 landmarks
pointset3_digest: 2 landmarks
pointset4_heart: 4 landmarks
pointset5_eyes: 11 landmarks
pointset6_skull_front: 8 landmarks
pointset7_skull_center: 7 landmarks
pointset8_skull_end: 6 landmarks
pointset9_brain: 10 landmarks


In [14]:
print(names.count('brain'))
print(names.count('fins'))

1
2


In [51]:
df[df['name'] == 'tc'].points_set.unique()

array(['pointset1_vert', 'pointset2_fins'], dtype=object)

In [49]:
df['name'].unique()

array(['jo', 'jvm', 'kk', 'tt', 'cs', 'tc', 'bew', 'vc', 'ra', 'ttt',
       'fins', 'kp', 'jf', 'brain'], dtype=object)

In [92]:
read_landmarks(data_path / '1064_pointset1_vert_JO.mps')

[array([410.68527946, 361.34097292, 837.83204157]),
 array([426.68527946, 318.24754659, 907.56993833]),
 array([424.81810551, 311.09290667, 963.36025574]),
 array([ 421.68527946,  308.14398353, 1020.22346386]),
 array([ 418.13282604,  311.98799983, 1076.01378127]),
 array([ 409.99999999,  318.95207945, 1136.04074874])]

In [148]:
read_landmarks(data_path / '1064_pointset1_vert_JO (1).mps')

[array([284.99999999, 262.78677778, 578.70023437]),
 array([291.18236118, 238.71703727, 618.7555187 ]),
 array([295.99999999, 227.14703661, 656.28763551]),
 array([300.99999999, 223.67768127, 692.55816857]),
 array([306.99999999, 221.15451376, 729.14409756]),
 array([311.99999999, 220.83911782, 766.04542249]),
 array([ 195.00365284,  366.0317758 , 1767.        ])]

In [60]:
filecmp.cmp(data_path / '1100_PointSet6_Skull_Front_RA.mps', data_path / '1100_PointSet6_Skull_Front_RA (1).mps')

True

In [31]:
[f for f in os.listdir(data_path) if os.path.isfile(f)]

[]

In [38]:
len([x for x in data_path.iterdir() if x.is_file()])

1816